# 1. eFLINT-2.0 features

Since eFLINT-2.0 it is possible to mix and match (type-)declarations, statements and queries. The term *phrase* is used to refer to an eFLINT fragment that is either a declaration, statement or a query. 

The `#include` and `#require` directives can be used to include files. The difference between 'include' and 'require' is that a `#require` directive is only executed when the included file was not included earlier in the current session. When a file is included more than once, all phrases within the file are executed a second time. This may result in unexpected outcomes, e.g. resulting in type-declarations or truth-assignments being overwritten unexpectedly. 

```
#include "example_includes.eflint". // file included for the first time
#require "example_includes.eflint". // not included again
```

Including files is not supported in the eFLINT notebooks and is therefore not demonstrated here.

In the remainder of this notebook we explore an example related to bidding in an action, demonstrating various of the features of eflint-2.0.

In [1]:
Fact bidder
Fact object Identified by Watch, Clock, Painting

Fact price Identified by Int
Fact min-price-of Identified by object * price.

+min-price-of(Watch, 100).
+min-price-of(Clock, 200).
+min-price-of(Painting, 400).

Fact bid Identified by bidder * object * price

+min-price-of(object("Clock"),price(200))
+min-price-of(object("Painting"),price(400))
+min-price-of(object("Watch"),price(100))


## 1.1 Type extensions

Since eFLINT-2.0 it is possible to write type-extensions that add additional clauses to types declared previously. (See `eflint3-features/1_clauses` for an explanation of exactly which clauses can be added by type-extensions.)

The following type-extension adds a derivation rule that automatically adds a bid from actor Alice to for every object at the minimal price.

In [2]:
Extend Fact bid 
  Derived from bid(Alice, min-price-of.object, min-price-of.price)

+bid(bidder("Alice"),object("Clock"),price(200))
+bid(bidder("Alice"),object("Painting"),price(400))
+bid(bidder("Alice"),object("Watch"),price(100))


The following fragment redefines the type `bid`, adding an additional field, a counter, keeping track of how many bids have been place on the object before the current bid.

In [3]:
Fact bids Identified by Int
Fact bid Identified by bidder * object * price * bids

~bid(bidder("Alice"),object("Clock"),price(200))
~bid(bidder("Alice"),object("Painting"),price(400))
~bid(bidder("Alice"),object("Watch"),price(100))


As the output above shows, replacing the original type declaration also removes the (derivation) clauses added by the earlier type-extension.

The following action describes the *power* of placing a bid on an object given a price.

In [4]:
Act place-bid Actor bidder Related to object, price
  Holds when bidder
  Conditioned by price > Max(Foreach bid: bid.price When bid.object == object)          // higher than current max
  Creates bid(bidder, object, price, Count(Foreach bid: bid When bid.object == object)) // increase counter

The auctioneer displays objects during the action, and sets the initial price.

In [5]:
Fact auctioneer
Fact display Identified by object

Act display-object Related to object
  Holds when auctioneer(actor)
  Terminates display, price
  Creates display()

## 1.2 Syncs with

Act-type and event-type declarations can have `Syncs with` clauses that cause instances of these types to *synchronise* with other actions/events. A `Syncs with` clauses consists of an instance expression, evaluating to one or more instances of an act- or event-type, possibly using `Foreach`. Whenever an action or event is triggered, any `Syncs with` clauses associated with the type of the action or event is evaluated. The resulting actions and events are also triggered. In this way, the triggering action or event can be considered to inherit the effects of the other actions/events.

In our example, the `Syncs with` clause is used to connect the physical connection of raising one's hand at an action is **qualified** as the power of placing a bid, but only when one is a bidder (and not, for example, the auctioneer) and when an object is on display. The price associated with that bid is always `10 * n+1` higher than the minimum price, where `n` is the number of bids previously placed on the object.

In [6]:
Act raise-hand 
  Holds when True // an actor can always raise their hand
  Syncs with place-bid(bidder = actor
                      ,object = object
                      ,price  = min-price-of.price + 10 * (1+Count(Foreach bid: bid When bid.object == object))
                      )
        When bidder(actor)
          && display(object)
          && (min-price-of.object == object)

The following code cells demonstrate the effects of synchronisation.

In [7]:
+bidder(Alice). +bidder(Bob). +bidder(Chloe).
+auctioneer(David).

display-object(David, Watch).

-display(object("Clock"))
-display(object("Painting"))
+bidder("Alice")
+bidder("Bob")
+bidder("Chloe")
+auctioneer("David")
+display(object("Watch"))


Executed transition: display-object(actor("David"),object("Watch"))



In [8]:
raise-hand(Alice).

+bid(bidder("Alice"),object("Watch"),price(110),bids(0))


Executed transition: raise-hand(actor("Alice"))
Executed transition: place-bid(bidder("Alice"),object("Watch"),price(110))



In [9]:
raise-hand(Bob).

+bid(bidder("Bob"),object("Watch"),price(120),bids(1))


Executed transition: raise-hand(actor("Bob"))
Executed transition: place-bid(bidder("Bob"),object("Watch"),price(120))



In [10]:
raise-hand(Alice).

+bid(bidder("Alice"),object("Watch"),price(130),bids(2))


Executed transition: raise-hand(actor("Alice"))
Executed transition: place-bid(bidder("Alice"),object("Watch"),price(130))



In [11]:
raise-hand(Chloe).

+bid(bidder("Chloe"),object("Watch"),price(140),bids(3))


Executed transition: raise-hand(actor("Chloe"))
Executed transition: place-bid(bidder("Chloe"),object("Watch"),price(140))



In [12]:
display-object(David, Painting).

-display(object("Watch"))
+display(object("Painting"))


Executed transition: display-object(actor("David"),object("Painting"))



In [13]:
raise-hand(Alice).

+bid(bidder("Alice"),object("Painting"),price(410),bids(0))


Executed transition: raise-hand(actor("Alice"))
Executed transition: place-bid(bidder("Alice"),object("Painting"),price(410))



In [14]:
raise-hand(Chloe).

+bid(bidder("Chloe"),object("Painting"),price(420),bids(1))


Executed transition: raise-hand(actor("Chloe"))
Executed transition: place-bid(bidder("Chloe"),object("Painting"),price(420))



In [15]:
raise-hand(Bob).

+bid(bidder("Bob"),object("Painting"),price(430),bids(2))


Executed transition: raise-hand(actor("Bob"))
Executed transition: place-bid(bidder("Bob"),object("Painting"),price(430))

